In [8]:
from trp import Document
import boto3

In [3]:
# Document
documentName = "C:/Users/hrint/Documents/Python2022/healthcare/extraction/Referral_Form_page-0001.jpg"

In [4]:
# Amazon Textract client
textract = boto3.client('textract')

c:\users\hrint\appdata\local\programs\python\python36\lib\site-packages\boto3\compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [58]:
# Call Amazon Textract
with open(documentName, "rb") as document:
    response = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS", ])

In [74]:
block = response["Blocks"]
if item["BlockType"] == "LINE":
        print ('\033[94m' +  item["Text"] + '\033[0m')

In [23]:
doc = Document(response)

In [49]:
for x in doc.pages[0].form.fields:
#     print( (x.key.content,x.value.content), end = "")
    for y in  x.key.content:
        print(y)

Phone:
Referred
On:
Phone:
City,
St
Zip:
City,
St
Zip:
Referred
To:
Fax:
Policy#:
Address:
DOB:
City,
St
Zip:
Address:
Referred
By:
Patient
Name:
Phone:
Fax:
Primary/Billing
Diagnosis:
Address:
Insurance:
Referral
Reason:
Attachments:


In [27]:
from tabulate import tabulate

In [32]:
dir(doc.pages[0].form.fields[0].key)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_block',
 '_confidence',
 '_content',
 '_geometry',
 '_id',
 '_text',
 'block',
 'confidence',
 'content',
 'geometry',
 'id',
 'text']

In [57]:
 (doc.pages[0].form.fields)


In [82]:
import json

import boto3
import io
from PIL import Image, ImageDraw
import json

def ShowBoundingBox(draw,box,width,height,boxColor):      
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline=boxColor)   

def ShowSelectedElement(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],fill=boxColor)  

# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['Id']))
    if 'Text' in block:
        print('    Detected: ' + block['Text'])
    print('    Type: ' + block['BlockType'])
   
    if 'Confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

    if block['BlockType'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['ColumnIndex']))
        print("        Row:" + str(block['RowIndex']))
        print("        Column Span:" + str(block['ColumnSpan']))
        print("        RowSpan:" + str(block['ColumnSpan']))    
    
    if 'Relationships' in block:
        print('    Relationships: {}'.format(block['Relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
    print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['EntityTypes'][0])
    
    if block['BlockType'] == 'SELECTION_ELEMENT':
        print('    Selection element detected: ', end='')

        if block['SelectionStatus'] =='SELECTED':
            print('Selected')
        else:
            print('Not selected')    
    
    if 'Page' in block:
        print('Page: ' + block['Page'])
    print()

def process_text_analysis():

    
    
    with open(documentName, "rb") as document:
        response = textract.analyze_document(
            Document={
                'Bytes': document.read(),
            },
            FeatureTypes=["FORMS", ])
    
    with open("response_sample2.json", "w") as fwriter:
        json.dump(response, fwriter)
    blocks=response['Blocks']
#     width, height =image.size    
    print ('Detected Document Text')
   
    # Create image showing bounding box/polygon the detected lines/text
    for block in blocks:
        DisplayBlockInformation(block)    
        draw=ImageDraw.Draw(image)

        # Draw bounding boxes for different detected response objects
        if block['BlockType'] == "KEY_VALUE_SET":
            if block['EntityTypes'][0] == "KEY":
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'red')
            else:
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'green')             
        if block['BlockType'] == 'TABLE':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'blue')
        if block['BlockType'] == 'CELL':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'yellow')
        if block['BlockType'] == 'SELECTION_ELEMENT':
            if block['SelectionStatus'] =='SELECTED':
                ShowSelectedElement(draw, block['Geometry']['BoundingBox'],width,height, 'blue')    
            
    # Display the image
    image.show()
    return len(blocks)


process_text_analysis()

Detected Document Text
Id: 32bf10cc-cdd1-4377-8f98-0b13da04a4fd
    Type: PAGE
    Relationships: [{'Type': 'CHILD', 'Ids': ['7597ffee-c96b-47cc-a194-65f5137032be', 'dd320cde-5070-4725-b1e8-9dc73ab1d13f', '1e545911-eb01-463d-be37-e9ac452e346a', 'f456c4e1-0634-4948-9b0a-1dda484450d3', '6a837514-39bf-4e60-8d9b-ac6362561307', '51a98ea0-2880-4175-847c-8b3eac02c3ad', 'd2a62b31-2234-4b39-8972-3068f2851548', '3edf7207-9a0e-4a0a-98ca-a741995d8875', '813dee46-d41d-4132-b034-e7a6b6ac5e00', '604d9298-b29f-4e5b-9bf2-20c51db2968b', 'c9a519ea-1189-4624-bddb-01d37e23be26', 'cbef816c-0a44-4e1d-85d7-fdacf027cd97', '6a6f75d7-099f-48a6-b5a5-2837ae69d15c', '368300f4-fb3c-4af0-8361-626ba57d2d38', '4ee4334d-737a-472f-909c-233decf26b8c', '475c9d5c-d861-41c0-93fd-2210a18867c4', 'a270722f-81cf-4085-8ec6-4272b6e8c882', 'c185d32e-c736-42f8-9b93-8ffa28940968', 'c6519b5a-bd86-4430-834d-e94652a16200', 'f2ab630c-57d3-4695-a4d4-c6c4e3e71b15', '8ca3a3c5-ef18-4f07-8a30-ba49622d64ca', 'a51891b9-3e12-4713-9165-de89f9b3f7

NameError: name 'image' is not defined

In [81]:
!pip install pillow
